In [1]:
from collections import OrderedDict

import pandas as pd
import numpy as np

from ipywidgets import HBox, VBox, RadioButtons, Label, Layout

from bqplot import DateScale, OrdinalScale, LinearScale, Lines, Bars, Axis, Figure
from bqplot.colorschemes import CATEGORY10, CATEGORY20, CATEGORY20c
from bqplot.interacts import FastIntervalSelector

import warnings
warnings.simplefilter('ignore')

In [2]:
yc_df = pd.read_csv('yc_time_series.csv', index_col=0, parse_dates=True)
yc_data = yc_df.dropna(axis=0).values
dates = yc_df.index.values
tenors = list(yc_df.columns)

In [3]:
# get 12 cross-section samples from yc_data
def slice_yield_curves(yc_data):
    num_rows = yc_data.shape[0]
    step = num_rows // 12
    return yc_data[::step, :] if step > 1 else yc_data

def pca(data):
    diffs = np.diff(data, axis=0)
    cov_mat = np.cov(diffs.T)
    
    # eigen decomposition of the cov matrix
    w, V = np.linalg.eig(cov_mat)

    # sort w, V based on eigenvalues desc
    sort_idx = np.argsort(w)[::-1]
    w, V = w[sort_idx], V[:, sort_idx]

    # de-normalize factors with sqrt of eigen values
    V = V * np.sqrt(w)

    # convert the 1st factors to be positive
    if np.mean(V[:, 0]) < 0:
        V[:, 0] *= -1

    # convert the 2nd factors to have positive slope
    if (V[-1, 1] - V[0, 1]) < 0:
        V[:, 1] *= -1
    if V[-1, 2] < 0:
        V[:, 2] *= -1

    return w, V

In [4]:
overflow = Layout(overflow_x='visible', overflow_y='visible')
status_label = Label()
status_label.layout.align_self = 'center'
status_label.layout.width = '400px'

#time series chart of yields with intsel
xs1, ys1 = DateScale(), LinearScale(min=0., max=0.06)
time_series = Lines(x=[], y=[], scales={'x': xs1, 'y': ys1}, colors=CATEGORY20c, labels=tenors, labels_visibility='label', apply_clip=False)
xax1 = Axis(scale=xs1, label='Dates', grid_lines='solid')
yax1 = Axis(scale=ys1, orientation='vertical', tick_format='.1%', grid_lines='solid', label_location='end', label_offset='-2ex')
intsel = FastIntervalSelector(scale=xs1, marks=[time_series])
ts_fig = Figure(marks=[time_series], axes=[xax1, yax1], interaction=intsel, layout=Layout(width='1525px', height='500px'))

#yield curve chart
xs2, ys2 = OrdinalScale(), LinearScale(min=0., max=0.06)
yc_lines = Lines(x=[], y=[], interpolation='basis', colors=CATEGORY20, scales={'x': xs2, 'y': ys2}, stroke_width=2)
xax2 = Axis(scale=xs2, label='Tenor', grid_lines='solid', label_location='end', label_offset='-1em')
yax2 = Axis(scale=ys2, orientation='vertical', label='Rate', tick_format='.1%', grid_lines='solid',  
            label_location='end', label_offset='-2ex')
yc_fig = Figure(marks=[yc_lines], axes=[xax2, yax2], 
                title='Yield Curves (12 selected from the above interval)', layout=Layout(width='550px', height='400px'))

#spectrum chart
xs3, ys3 = OrdinalScale(), LinearScale(micolorn=0., max=1)
spectrum_bar = Bars(x='F1 F2 F3'.split(), y=[], colors=CATEGORY10, scales={'x': xs3, 'y': ys3})
xax3, yax3 = Axis(scale=xs3, grid_lines='none'), Axis(scale=ys3, orientation='vertical', tick_format='.0%', grid_lines='solid')
spectrum_fig = Figure(marks=[spectrum_bar], axes=[xax3, yax3], title='Variance Explained',
                      layout=Layout(width='400px', height='400px'))

#pca factor chart
xs4, ys4 = OrdinalScale(), LinearScale()
#add a zero line for reference
zero_line = Lines(x=[], y=[], colors=['#ccc'], scales={'x': xs4, 'y': ys4}, stroke_width=2)
factor_lines = Lines(x=[], y=[], interpolation='basis', colors=CATEGORY10, scales={'x': xs4, 'y': ys4}, stroke_width=3.5, 
                     labels=['PCA Factor ' + str(i+1) for i in range(3)], display_legend=True)
xax4 = Axis(scale=xs4, label='Tenor', grid_lines='solid', label_location='end', label_offset='-1em')
yax4 = Axis(scale=ys4, orientation='vertical', tick_format='.4f', grid_lines='solid')
factor_fig = Figure(marks=[zero_line, factor_lines], 
                    axes=[xax4, yax4], title='First 3 PCA factors', layout=Layout(width='600px', height='400px'))

#link the widgets/charts
def do_pca(change):
    selected = change.get('new', None) if change else None
    if selected is None:
        s, e = 0, yc_data.shape[0] - 1
    else:
        s, e = selected[0], selected[-1]

    ts_fig.title = 'Constant Maturity Treasury Yield Curve (From ' + pd.to_datetime(dates[s]).strftime('%m/%d/%Y') + \
                   ' To ' +  pd.to_datetime(dates[e]).strftime('%m/%d/%Y') + ')'
    yc_data_slice = yc_data[s:e, :]
    yc_lines.y = slice_yield_curves(yc_data_slice)
    var, factors = pca(yc_data_slice)
    factor_lines.y = factors[:, :3].T
    var_expl = var / np.sum(var)
    spectrum_bar.y = var_expl[:3]
    spectrum_fig.title = 'Variance Explained: ' + '{:.0%}'.format(np.sum(var_expl[:3]))

time_series.observe(do_pca, 'selected')

xs2.domain = xs4.domain = tenors
ts_fig.title = 'Constant Maturity Treasury Yield Curve (From ' + pd.to_datetime(dates[0]).strftime('%m/%d/%Y') + \
               ' To ' +  pd.to_datetime(dates[-1]).strftime('%m/%d/%Y') + ')'

with time_series.hold_sync():
    time_series.x, time_series.y = dates, yc_data.T

with yc_lines.hold_sync():
    yc_lines.x = tenors
    yc_lines.y = yc_data[::yc_data.shape[0]//12, :]

with zero_line.hold_sync():
    zero_line.x = np.array([tenors[0], tenors[-1]])
    zero_line.y = [0, 0]

factor_lines.x = tenors

do_pca(None)
VBox([ts_fig, HBox([yc_fig, spectrum_fig, factor_fig], layout=overflow), status_label], layout=overflow)